In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('outbreaks.csv')

In [3]:
df.head()

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
0,1998,January,California,Restaurant,NaN,NaN,NaN,NaN,NaN,20,0.0,0.0
1,1998,January,California,NaN,Custard,NaN,NaN,NaN,NaN,112,0.0,0.0
2,1998,January,California,Restaurant,NaN,NaN,NaN,NaN,NaN,35,0.0,0.0
3,1998,January,California,Restaurant,"Fish, Ahi",NaN,Scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"Lasagna, Unspecified; Eggs, Other",NaN,Salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0


In [4]:
df = df[df['Food'].notnull()][df['Species'].notnull()]

/home/jackdougherty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [5]:
df.head()

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
3,1998,January,California,Restaurant,"Fish, Ahi",NaN,Scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"Lasagna, Unspecified; Eggs, Other",NaN,Salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0
7,1998,January,California,Restaurant,"Stuffing, Unspecified; Sandwich, Turkey",NaN,Salmonella enterica,Enteritidis,Confirmed,4,3.0,0.0
8,1998,January,Colorado,Restaurant,"Salad, Unspecified",NaN,Virus,NaN,Suspected,21,NaN,NaN
9,1998,January,Colorado,Restaurant,"Peppers, Chili",NaN,Virus,NaN,Suspected,3,NaN,0.0


In [13]:
species_string_list = df['Food'].unique()

In [27]:
species_string_list_clean = []

for species_list in species_string_list:
    species_list = species_list.split(',')
    for species in species_list:
        species = species.strip()
        species_string_list_clean.append(species)

speciest_string_list_clean = list(set(species_string_list_clean))

In [29]:
species_string_list_clean

['Fish',
 'Ahi',
 'Lasagna',
 'Unspecified; Eggs',
 'Other',
 'Stuffing',
 'Unspecified; Sandwich',
 'Turkey',
 'Salad',
 'Unspecified',
 'Peppers',
 'Chili',
 'Ethnic Style',
 'Unspecified',
 'Greek Salad',
 'Ground Beef',
 'Cheeseburger',
 'Pasta',
 'Unspecified',
 'Sauces',
 'Unspecified; Tuna',
 'Unspecified',
 'Sandwich',
 'Chicken',
 'Chicken',
 'Fried',
 'Salmon',
 'Raw; Tuna',
 'Raw',
 'Chicken',
 'Unspecified',
 'Fish',
 'Grouper',
 'Unspecified',
 'Fish',
 'Mahi Mahi',
 'Chicken Salad',
 'Burrito',
 'Chicken',
 'Cake',
 'Tiramisu; Eggs',
 'Other',
 'Chicken Fried Rice',
 'Dips',
 'Unspecified',
 'Eggnog',
 'Homemade',
 'Sausage',
 'Dried',
 'Homemade Cheese',
 'Unspecified',
 'Pasta',
 'Unspecified; Meatballs',
 'Turkey Salad',
 'Beef',
 'Unspecified',
 'Enchilada',
 'Chicken',
 'Onion Dip',
 'Refried Beans',
 'Unspecified; Peppers',
 'Chili',
 'Cream Cheese',
 'French Toast',
 'Unspecified; Shell Eggs',
 'Scrambled Hard',
 'Stir-Fry',
 'Unspecified; Lettuce-Based Salads Unsp